## Synonym of Baseline Object
Q: Why do we need to list the synonyms of Baseline Object?  
A: We expect a fuzzy matching method.

In [2]:
org_base = {'sh': 60, 'ID': 40, 'sed': 38, 'malware': 18, '1526': 13, '/etc/rc.local': 9, '1527': 9, '8192*1024 bytes': 7, 'status:0': 7, '23.236.66.13:50050': 6, 'status:2': 4, 'Permission:0700': 3, 'Permission:022': 3, 'status:1': 2, 'Sleep Duration': 2, '/proc/net/dev': 2, '/proc/stat': 2, 'uname': 1, '/proc/self/exe': 1, '"/prober"': 1, '/etc/rc.d/rc.local': 1, '/etc/init.d/boot.local': 1, '1528': 1, '1529': 1, '/dev/urandom': 1, 'NIC': 1, 'Timestamp': 1, '/sys/devices/system/cpu/online': 1}

synonyms = {
    'ID': ['process id', 'pid'],
    'NIC': ['']
}

def get_baseline_for_eval(org_base: list) -> list:
    return

從 `triplate_statistics.xlsx` 的 excel 檔建立 baseline，只取出 file, net 的類型，注意 other 的 uname 也要加進來。  
ver4 dst_node set with length: `27` 

In [19]:
import re
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
from typing import Dict, List, get_type_hints
from IPython.display import display

In [4]:
df = pd.read_excel(open('triplate_statistics.xlsx', 'rb'), sheet_name='ver4', index_col=0)  
org_base = set(df['dst_node'])
print(len(org_base))
# org_base

27


In [5]:
# basedata 是要送進評估系統裡的dict物件
base_data = {
    'text':[],
    'type':[]
}
for i,row in df.iterrows():
    if row.dst_node not in base_data['text']:
        if row.dst_node == 'uname' or row['type.1'] in ['file','net']: # 只取 file, net 和 uname
            base_data['text'].append(re.sub(r"\"", '', row.dst_node))
            base_data['type'].append(row['type.1'])
pd.DataFrame(base_data).sort_values(by='type')

,text,type
1,/proc/self/exe,file
2,/prober,file
3,/etc/rc.local,file
4,/etc/rc.d/rc.local,file
5,/etc/init.d/boot.local,file
6,/dev/urandom,file
9,/proc/net/dev,file
10,/sys/devices/system/cpu/online,file
11,/proc/stat,file
7,NIC,net


### 使用 Dofloo 的文章來測試

In [6]:
report_fir_path = '../C parse report/sentence csvs/'

def get_all_filenames(dir: str='./') -> list:
    ''' traverse root directory, and list directories as dirs and files as files. Return filenames in rootdir. '''
    files = [f for f in listdir(dir) if isfile(join(dir, f))]
    files.sort()
    return files

dofloo_report_names = [f for f in get_all_filenames(report_fir_path) if f.startswith('Dofloo')]
dofloo_report_dfs = [pd.read_csv(f"{report_fir_path}{f}") for f in dofloo_report_names]
dofloo_report_names

['Dofloo-BleepingComputer.csv',
 'Dofloo-MalwareMustDie.csv',
 'Dofloo-Securityaffairs.csv',
 'Dofloo-SyscallParty.csv',
 'Dofloo-Trendmicro.csv']

In [7]:
dofloo_report_dfs[0]

,Number,Content
0,"""Sentence 1","exposed docker apis abused by ddos, cryptojack..."
1,"""Sentence 2",the dofloo (aka aesddos) malware was first det...
2,"""Sentence 3",misconfigured docker services being abused is ...
3,"""Sentence 4",allowing external access — whether intentional...
4,"""Sentence 5",query to list all available containers\nthe at...
5,"""Sentence 6",the dofloo botnet malware is subsequently depl...
6,"""Sentence 7",system information is also collected by the tr...
7,"""Sentence 8",dofloo (aesddos) malware executing remote shel...
8,"""Sentence 9",previous dofloo and docker attacks\ndofloo was...
9,"""Sentence 10",exposed docker apis were abused by other malic...


In [8]:
res = re.search(r"[^a-z]sh[^a-z]", "www sh www")
res

<re.Match object; span=(3, 7), match=' sh '>

In [9]:
class ReportEvalModel:
    def __init__(self, baseline: Dict[str, str], sentences: pd.DataFrame, reportname:str=''):
        self.baseline = baseline
        self.sentences = sentences # a df
        self.reportname = reportname
        cols = baseline['text']
        self.match_tbl = pd.DataFrame([[0]*len(cols)]*len(sentences), columns=cols)
        self.match_baseline = set()
        self.match()

    def match(self):
        '''find baseline in sentences of report. Fill matching result in self.match_tbl.'''
        sentence_list = self.sentences['Content']
        for idx_sent, sent in enumerate(sentence_list):
            # print(idx_sent, sent)
            # for idx_base, base in enumerate(self.baseline.text):
            for idx_base, (base, type) in enumerate(zip(self.baseline['text'], self.baseline['type'])):
                # find baseline in a sentence
                if self.find_baseline(str(sent), base, type) != -1:
                    self.match_tbl.loc[idx_sent,base] = 1 # mark as found
                    self.match_baseline.add(base)
        pass

    def find_baseline(self, sentence: str, baseline: str, baseline_type: str='file'):
        '''not found will return -1'''
        if baseline_type == 'net':
            return sentence.find(baseline) # NIC, ip, port 先不做特別處理
        elif baseline in ['sed','sh']:
            # ^sh, 'sh', <sh>, www sh www (沒測過)
            if re.search(r"[^a-z]sh[^a-z]", sentence) or re.search(r"^sh[^a-z]", sentence):
                return 0
            if re.search(r"[^a-z]sed[^a-z]", sentence) or re.search(r"^sed[^a-z]", sentence):
                return 0
            return -1
        else:
            # 一般檔案就直接比對
            return sentence.find(baseline)
    
    def get_match_sentences(self) -> pd.DataFrame:
        result = self.match_tbl.copy()
        result['match'] = result.sum(axis=1)
        return result
    
    def get_match_baselines(self) -> set:
        return self.match_baseline

rem = ReportEvalModel(base_data, dofloo_report_dfs[4])

In [10]:
# er.match_tbl
rem.get_match_sentences()

,uname,/proc/self/exe,/prober,/etc/rc.local,/etc/rc.d/rc.local,/etc/init.d/boot.local,/dev/urandom,NIC,23.236.66.13:50050,/proc/net/dev,/sys/devices/system/cpu/online,/proc/stat,match
0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,1,1,0,0,0,0,0,0,3
5,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,1,0,0,0,0,0,0,0,1,0,1,3
7,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
rem.get_match_baselines()

{'/etc/init.d/boot.local',
 '/etc/rc.d/rc.local',
 '/etc/rc.local',
 '/proc/net/dev',
 '/proc/self/exe',
 '/proc/stat'}

In [15]:
class FamilySet:
    '''一個 malware family report 集合'''
    def __init__(self, familyname:str, baseline:Dict[str, str]):
        self.familyname = familyname
        self.baseline = baseline
        self.rem_lst: List[ReportEvalModel] = [] # list of ReportEvalModel under this family

    def add_rem(self, sentences: pd.DataFrame, reportname: str=''):
        '''新增 report 到 FamilySet 中，需傳入文本和報告名稱，會沿用 FamilySet 的 baseline.'''
        rem = ReportEvalModel(self.baseline, sentences, reportname)
        self.rem_lst.append(rem)

    def show_result(self):
        '''print and return result table. baselinse(x) * report(y).'''
        column_names = ['report_name','ttl_match']
        column_names.extend(self.baseline['text'])
        result_tbl = pd.DataFrame([[0]*len(column_names)]*len(self.rem_lst), columns=column_names)
        for i,rem in enumerate(self.rem_lst):
            match_baselines = rem.get_match_baselines() # 這篇報告含有哪些 baseine:set
            result_tbl.loc[i,'report_name'] = rem.reportname
            result_tbl.loc[i,'ttl_match'] = len(match_baselines)
            for b in match_baselines:
                result_tbl.loc[i,b] = 1
            print(rem.reportname, match_baselines)
        display(result_tbl)
        return result_tbl

fset = FamilySet('Dofloo', base_data)
for i,rdf in enumerate(dofloo_report_dfs):
    # rem = ReportEvalModel(base_data, rdf, dofloo_report_names[i])
    fset.add_rem(rdf, dofloo_report_names[i])
result_tbl = fset.show_result()
# result_tbl.to_excel('FamilySet_Dofloo.xlsx', index=False)

Dofloo-BleepingComputer.csv set()
Dofloo-MalwareMustDie.csv set()
Dofloo-Securityaffairs.csv {'/etc/rc.d/rc.local', '/etc/rc.local'}
Dofloo-SyscallParty.csv {'uname', '/proc/self/exe', '/proc/net/dev', '/etc/init.d/boot.local', '/etc/rc.local'}
Dofloo-Trendmicro.csv {'/proc/self/exe', '/proc/net/dev', '/proc/stat', '/etc/init.d/boot.local', '/etc/rc.d/rc.local', '/etc/rc.local'}


,report_name,ttl_match,uname,/proc/self/exe,/prober,/etc/rc.local,/etc/rc.d/rc.local,/etc/init.d/boot.local,/dev/urandom,NIC,23.236.66.13:50050,/proc/net/dev,/sys/devices/system/cpu/online,/proc/stat
0,Dofloo-BleepingComputer.csv,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dofloo-MalwareMustDie.csv,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dofloo-Securityaffairs.csv,2,0,0,0,1,1,0,0,0,0,0,0,0
3,Dofloo-SyscallParty.csv,5,1,1,0,1,0,1,0,0,0,1,0,0
4,Dofloo-Trendmicro.csv,6,0,1,0,1,1,1,0,0,0,1,0,1


### 印出/存檔 Report 中的 matched sentence
- matched baseline
- sentence 內文
- sentence number [1~n] 作為排序依據
- subject of sentnece
- verb of sentnece
- object of sentnece (等於 matched baseline)

In [100]:
def save_match_sent_to_excel(fset: FamilySet):
    columns = ['baseline','sentence number','sentence']
    output_xlsx = 'report_match_result.xlsx'
    with pd.ExcelWriter(output_xlsx, engine='openpyxl') as writer:
    
        # 遍歷報告，一個 rem: ReportEvalModel 代表一篇報告
        for rem in fset.rem_lst:
            if len(rem.get_match_baselines()) == 0: # 如果報告中無 baseline 跳過不紀錄
                continue
            sheet_data = [] # shape = columns_len * match_sent_len
            sent_df = rem.get_match_sentences()
            sent_content_lst = rem.sentences['Content'] # 報告中的每個句子

            # 遍歷每個含有 baseline 的句子
            for sid, row in sent_df.iterrows():
                if row['match'] == 0:
                    continue
                row.drop('match', inplace=True)
                row = row[row > 0]
                # print(row)
                # 考量每個句子可能含有多個 baseline，故需寫成多行
                for i, v in row.iteritems():
                    insert_data = dict().fromkeys(columns)
                    insert_data['baseline'] = i
                    insert_data['sentence number'] = sid + 1
                    insert_data['sentence'] = sent_content_lst[sid].strip()
                    sheet_data.append(insert_data)
            # 輸出 sheet
            print(rem.reportname, len(sheet_data))
            output_sheet = pd.DataFrame(sheet_data)
            styler = Styler(horizontal_alignment='left', vertical_alignment='top')
            StyleFrame(output_sheet, styler).to_excel(writer, sheet_name=rem.reportname, index=False) #.save()
    return

# 此行會執行並複寫 excel，執行完後要手動調整行距格式，小心使用
# save_match_sent_to_excel(fset)

Dofloo-Securityaffairs.csv 2
Dofloo-SyscallParty.csv 7
Dofloo-Trendmicro.csv 6


以下是測試寫入 excel 的 code

In [58]:
columns = ['baseline','sentence number','sentence']
sheet_data = [] # shape = columns_len * match_sent_len
# matched_series = fset.rem_lst[4].get_match_sentences()['match'] > 0 # 含有 baseline 的句子 id
# sent_idxlst = matched_series[matched_series].index.tolist()

sent_df = fset.rem_lst[4].get_match_sentences()
sent_content_lst = fset.rem_lst[4].sentences['Content']
# 遍歷每個含有 baseline 的句子
for sid, row in sent_df.iterrows():
    if row['match'] == 0:
        continue
    row.drop('match', inplace=True)
    row = row[row > 0]
    # print(row)
    # 考量每個句子可能含有多個 baseline，故需寫成多行
    for i, v in row.iteritems():
        insert_data = dict().fromkeys(columns)
        insert_data['baseline'] = i
        insert_data['sentence number'] = sid + 1
        insert_data['sentence'] = sent_content_lst[sid].strip()
        sheet_data.append(insert_data)
# print(sheet_data)

In [30]:
fset.rem_lst[4].sentences

,Number,Content
0,"""Sentence 1",backdoor.linux.dofloo.ab\n\naliases:\nbackdoor...
1,"""Sentence 2",it executes commands from a remote malicious u...
2,"""Sentence 3","technical details\n\nfile size: 1,001,465 byte..."
3,"""Sentence 4",NaN
4,"""Sentence 5",\ninstallation\n\nthis backdoor adds the follo...
5,"""Sentence 6",\nthis backdoor executes the following command...
6,"""Sentence 7",\nthis backdoor gathers the following data:\n\...
7,"""Sentence 8",\nthis backdoor opens the following files:\n\n...
8,"""Sentence 9",\nsolution\n\nminimum scan engine: 9.850\nfirs...
9,"""Sentence 10",you may opt to simply delete the quarantined f...


In [92]:
# 這個 style 很醜 但好改
from styleframe import StyleFrame, Styler
output_xlsx = 'report_match_result.xlsx'
output_sheet = pd.DataFrame(sheet_data)
styler = Styler(horizontal_alignment='left', vertical_alignment='top')
StyleFrame(output_sheet, styler).to_excel('test.xlsx', index=False).save()
# https://stackoverflow.com/questions/17326973/is-there-a-way-to-auto-adjust-excel-column-widths-with-pandas-excelwriter

In [83]:
from pandas.io.formats.excel import ExcelFormatter
with pd.ExcelWriter('test.xlsx', engine="xlsxwriter") as writer:
    cell_format = writer.book.add_format()
    cell_format.set_font_color('green')
    
    writer.book.formats[0].set_text_wrap()  # update global format with text wrap
    output_sheet.to_excel(writer, index=False)